# Introduction to Automation with LangChain, Generative AI, and Python
**2.6: Conversation Buffer Window Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

We previously saw that we could build up an LLM chat client by building an ever-increasing script of what the human and AI said in the conversation. We constantly add the human response and wait to see what the AI will respond to next. This cycle continues as long as the chat.

This ever-increasing chat memory is a typical pattern for LLMs, and as a result, LangChain has several predefined Python objects that allow you to implement this sort of memory-based chatbot.

## Creating a Chat Conversation

ConversationChain in LangChain is a framework designed to facilitate the development and management of conversational AI systems. It primarily orchestrates the interaction between different components, such as the dialogue management system, language models, and various types of memory that retain information from the conversation to improve response relevance and coherence.

In this context, memory types play a crucial role. They help the system remember and utilize past interactions to maintain context and enhance the continuity of the conversation. This system frees us from tracking the conversation as performed manually in the pervious section.

We first examine ConversationBufferWindowMemory is a dynamic memory model designed to manage the flow of a dialogue by retaining a record of the last K interactions within a conversation. This method ensures that the memory buffer maintains a manageable size, avoiding overflow and performance issues that can arise with an excessively large interaction history. By focusing on the most recent interactions, this memory type effectively creates a "sliding window" that continuously updates as new exchanges occur, allowing for contextually relevant responses while efficiently managing system resources. This approach is particularly beneficial in applications where maintaining an immediate and context-aware dialogue is crucial, such as in customer service bots or interactive learning tools.

The provided code snippet demonstrates how to set up and use a conversational AI system using the LangChain framework, specifically focusing on maintaining a concise memory of recent interactions. The ConversationChain class orchestrates the conversation flow, integrating various components such as language model, memory, and prompt formatting.

Firstly, several modules are imported, including ConversationChain for managing the conversation, ConversationBufferWindowMemory for handling the memory of recent interactions, and ChatOpenAI to interface with OpenAI's language models. The PromptTemplate is used to define the structure of the prompts sent to the language model.

In the setup, MODEL specifies the particular version of the language model (GPT-3.5 Turbo), and TEMPLATE outlines the format of the conversation, integrating previous dialogue history and the current user input into the prompt. This structured prompt is then used to create a PromptTemplate object.

The begin_conversation function initializes the language model and sets up the memory to store the last five interactions (k=5). This sliding window of memory helps keep the conversation relevant and efficient by focusing on the most recent exchanges. The ConversationChain object is created with the prompt template, the language model, and the memory buffer, and it's ready to process conversation inputs.

The converse function takes a conversation object and a user prompt to produce responses. It uses the conversation object's invoke method, which applies the prompt template, consults the memory, and generates a response based on the current and recent dialogue context.

Overall, this setup is optimized for creating a responsive and context-aware conversational AI that doesn't overload with too much past information, maintaining relevancy and efficiency in ongoing interactions.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation():
    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.7},
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory(k=5)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    print(f"***Human: {prompt}")
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [2]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

conversation = begin_conversation()
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

***Human: Hello, what is my name?


```
Unfortunately, I don't actually know your name since we haven't met before. I'm Claude, an AI assistant created by Anthropic.
```

***Human: Oh sorry, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic to be helpful, honest and harmless. Please let me know if you have any questions or if there is anything I can assist you with.
```

***Human: What is my name?


```
Your name is Jeff.
```

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [3]:
def chat(conversation, prompt):
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [4]:
conversation = begin_conversation()
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


***Human: What is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me during our conversation.
```

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you Jeff! I've updated my knowledge that your name is Jeff. Please let me know if you have any other questions - I'm here to help.
```

***Human: What is my name?


```
Your name is Jeff, based on your introduction earlier in our conversation.
```

***Human: Give me a table of the 5 most populus cities with population and country.


```
Here is a table of the 5 most populous cities in the world with their population and country:

| City | Population | Country |
| --- | --- | --- |
| Tokyo | 37,468,000 | Japan |
| Delhi | 29,398,000 | India |
| Mexico City | 21,581,000 | Mexico |
| São Paulo | 21,846,000 | Brazil |
| Mumbai | 20,961,000 | India |
```

Note: The population figures are estimates for the metropolitan area populations as of 2022, and may vary slightly depending on the source.

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

def begin_conversation_insurance():
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.7},
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory(k=5)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance()
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

***Human: What is my name?


I'm afraid I cannot provide your name, as I do not have access to personal information about you. As a helpful agent focused on life insurance, I can only provide general information related to life insurance policies and coverage. Please feel free to ask any questions you may have about life insurance.

***Human: Okay, then let me introduce myself, my name is Jeff


```
I apologize, but as an AI assistant focused solely on providing helpful information about life insurance, I cannot engage in casual conversation or learn personal details about you. My role is to answer questions specifically related to life insurance products, policies, coverage, and related topics.

If you have any questions pertaining to life insurance, I would be happy to assist you. Otherwise, I must refrain from further dialogue beyond the scope of life insurance information.
```

***Human: What is my name?


```
As I mentioned before, I do not actually have access to personal information about you or any other users. My role is limited to providing helpful information specifically about life insurance products and policies.

While I understand the desire to have a more personal interaction, I am an AI assistant created to focus solely on the topic of life insurance. I cannot learn, store or confirm any personal details like names.

Please feel free to ask me any questions related to life insurance, and I will do my best to provide accurate and useful information within that scope. However, I cannot engage in casual conversation or confirm personal details outside of the life insurance domain.
```

***Human: What is your favorite programming language?


```
As an AI assistant focused on providing helpful information about life insurance, I do not actually have personal preferences or knowledge about programming languages. My role is to answer questions and provide information specifically related to life insurance products, policies, coverage options, and related topics within that domain.

I apologize, but I cannot engage in discussions or answer questions outside of the life insurance subject area, as that is beyond the scope of my designed capabilities. Please feel free to ask me any other questions you may have pertaining to life insurance, and I will do my best to provide accurate and informative responses.
```

***Human: What is the difference between a term and whole life policy?


```
The primary difference between term life insurance and whole life insurance lies in the duration of coverage and the way the policies accumulate cash value.

Term Life Insurance:
- Provides temporary coverage for a specific time period, typically 10, 20, or 30 years
- Offers only a death benefit with no cash value component
- Premiums are generally lower than whole life insurance
- Coverage ends at the end of the term unless the policy is renewed or converted

Whole Life Insurance:
- Provides lifelong coverage as long as premiums are paid
- Builds up a cash value component that can be borrowed against or withdrawn
- Premiums are typically higher than term life insurance
- The cash value grows tax-deferred over time
- Offers a guaranteed death benefit payout

In summary, term life insurance provides temporary protection at a lower cost, while whole life insurance offers permanent coverage with a cash value component, but at a higher premium cost. The choice between the two depends on an individual's specific needs, budget, and long-term financial goals.
```

## Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [6]:
print(conversation.memory.buffer)

Human: What is my name?
AI: I'm afraid I cannot provide your name, as I do not have access to personal information about you. As a helpful agent focused on life insurance, I can only provide general information related to life insurance policies and coverage. Please feel free to ask any questions you may have about life insurance.
Human: Okay, then let me introduce myself, my name is Jeff
AI: ```
I apologize, but as an AI assistant focused solely on providing helpful information about life insurance, I cannot engage in casual conversation or learn personal details about you. My role is to answer questions specifically related to life insurance products, policies, coverage, and related topics.

If you have any questions pertaining to life insurance, I would be happy to assist you. Otherwise, I must refrain from further dialogue beyond the scope of life insurance information.
```
Human: What is my name?
AI: ```
As I mentioned before, I do not actually have access to personal information 

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationBufferWindowMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [7]:
import uuid

conversation = begin_conversation()
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(10):
  chat(conversation, f"You need to remember fact #{i}, which is {str(uuid.uuid4())}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
chat(conversation, "OMG, you had one job!")

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I'm Claude, an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything specific I can assist you with.
```

***Human: You have ONE JOB! Remember that my favorite color is blue.


```
Understood, Jeff! I will remember that your favorite color is blue.
```

***Human: Do you remember my name?


...

```
No, I'm afraid I don't have any information about your name in our conversation history.
```

***Human: Do you remember my favorite color?


```
No, I don't have any information about your favorite color in our conversation history.
```

***Human: OMG, you had one job!


```
I do not actually have personal information about you stored. As an AI assistant, I respond based on our conversation history. If you did not provide details like your name or favorite color, I do not have that information to remember. My role is to be helpful, harmless, and honest within the context of each conversation.
```

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [8]:
print(conversation.memory.buffer)

Human: You need to remember fact #8, which is 03417497-4eef-4335-919f-3d2351b49f5b
AI: ```
Understood, fact #8 is 03417497-4eef-4335-919f-3d2351b49f5b.
```
Human: You need to remember fact #9, which is 44080eb4-2f53-496a-b103-e56898797822
AI: ```
Confirmed, fact #9 is 44080eb4-2f53-496a-b103-e56898797822.
```
Human: Do you remember my name?
AI: ```
No, I'm afraid I don't have any information about your name in our conversation history.
```
Human: Do you remember my favorite color?
AI: ```
No, I don't have any information about your favorite color in our conversation history.
```
Human: OMG, you had one job!
AI: ```
I do not actually have personal information about you stored. As an AI assistant, I respond based on our conversation history. If you did not provide details like your name or favorite color, I do not have that information to remember. My role is to be helpful, harmless, and honest within the context of each conversation.
```
